In [2]:
import boto3
import os
import sagemaker

bucket = 'di-lab-sandbox'
prefix = 'shiba/sagemaker-minimum'

In [3]:
boto3.resource('s3').Bucket(bucket).upload_file('/Users/ko.shibata/Desktop/train.txt', os.path.join(prefix, 'train', 'train.txt'))
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, 'train.txt')
print('uploaded training data location: {}'.format(s3_train_data))

uploaded training data location: s3://di-lab-sandbox/shiba/sagemaker-minimum/train/train.txt


In [4]:
boto3.resource('s3').Bucket(bucket).upload_file('/Users/ko.shibata/Desktop/test.txt', os.path.join(prefix, 'test', 'test.txt'))
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, prefix, 'test.txt')
print('uploaded test data location: {}'.format(s3_test_data))

uploaded test data location: s3://di-lab-sandbox/shiba/sagemaker-minimum/test/test.txt


In [48]:
image_name = '149721313133.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-example:0.1'
sagemaker_role = 'arn:aws:iam::149721313133:role/service-role/AmazonSageMaker-ExecutionRole-20181015T165338'
output_path = 's3://' + bucket + '/' + prefix + '/output'
boto3_session = boto3.Session(profile_name='di', region_name="ap-northeast-1")

sample_estimater = sagemaker.estimator.Estimator(image_name,
    sagemaker_role,
    train_instance_count=1,
    train_instance_type='ml.m5.xlarge',
    output_path=output_path,
    sagemaker_session=sagemaker.Session(boto_session = boto3_session))

In [49]:
hyperparams = {
    'my_var': 100,
    'my_var2': 'my_text' 
}
sample_estimater.set_hyperparameters(**hyperparams)

In [50]:
fit_input = {'train': s3_train_data, 'test': s3_test_data}
sample_estimater.fit(fit_input)

INFO:sagemaker:Creating training-job with name: sagemaker-example-2018-10-16-23-09-08-316


2018-10-16 23:09:08 Starting - Starting the training job...
Launching requested ML instances......
Preparing the instances for training...
2018-10-16 23:10:43 Downloading - Downloading input data
2018-10-16 23:10:51 Training - Training image download completed. Training in progress.
2018-10-16 23:11:06 Uploading - Uploading generated training model.
----/opt/ml/
input
model
output
----/opt/ml/input/
config
data
----/opt/ml/input/data/
train-manifest-1
test
train-manifest
test-manifest-1
train
test-manifest
----see train data
['train text\n']
----see test data
['test text\n']
----/opt/ml/input/config/
inputdataconfig.json
init-config.json
hyperparameters.json
metric-definition-regex.json
resourceconfig.json
----see hyperparameters data
{'my_var2': 'my_text', 'my_var': '100'}
----see resourceconfig data
{'current_host': 'algo-1', 'hosts': ['algo-1'], 'network_interface_name': 'ethwe'}

2018-10-16 23:11:11 Completed - Training job completed
Billable seconds: 29


In [15]:
job_name = sample_estimater._current_job_name
job_name

'sagemaker-example-2018-10-16-00-55-08-788'

In [19]:
# not used
import time
sample_estimator_2 = sagemaker.estimator.Estimator.attach(
    training_job_name='sagemaker-example-2018-10-16-00-55-08-788',
    sagemaker_session=sagemaker.Session(boto_session = boto3_session)
)
endpoint_name = 'knn-ml-m4-xlarge-%s'% (str(time.time()).replace('.','-'))
sample_estimator_2.deploy(initial_instance_count=1,
        instance_type='ml.m4.xlarge',
        endpoint_name=endpoint_name
)

2018-10-16 00:55:09 Starting - Preparing the instances for training
2018-10-16 00:56:54 Downloading - Downloading input data
2018-10-16 00:57:01 Training - Training image download completed. Training in progress.
2018-10-16 00:57:21 Uploading - Uploading generated training model
2018-10-16 00:57:26 Completed - Training job completed----/opt/ml/
input
model
output
----/opt/ml/input/
data
config
----/opt/ml/input/data/
test-manifest-1
train-manifest-1
test-manifest
train-manifest
test
train
----see train data
['train text\n']
----see test data
['test text\n']
----/opt/ml/input/config/
resourceconfig.json
hyperparameters.json
init-config.json
metric-definition-regex.json
inputdataconfig.json
----see hyperparameters data
{'my_var2': 'my_text', 'my_var': '100'}


INFO:sagemaker:Creating model with name: sagemaker-example-2018-10-16-01-04-54-107


Billable seconds: 33


INFO:sagemaker:Creating endpoint with name knn-ml-m4-xlarge-1539651894-045104


-------------------------------------------------------------------------------------------------------------------------

KeyboardInterrupt: 

In [21]:
model_path = output_path + '/' + job_name + '/output/model.tar.gz'
sample_model = sagemaker.model.Model(
    model_data=model_path,
    image=image_name,
    role=sagemaker_role,
    sagemaker_session=sagemaker.Session(boto_session = boto3_session)
)

In [47]:
endpoint_name = 'knn-ml-m4-xlarge-%s'% (str(time.time()).replace('.','-'))
sample_model.deploy(initial_instance_count=1,
        instance_type='ml.m4.xlarge',
        endpoint_name=endpoint_name
)

INFO:sagemaker:Creating model with name: sagemaker-example-2018-10-16-01-15-54-364


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: 1 validation error detected: Value 'local' at 'productionVariants.1.member.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.p3.16xlarge, ml.m5.large, ml.t2.xlarge, ml.p2.16xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.c5.4xlarge, ml.c4.8xlarge, ml.c4.large, ml.c5.large, ml.c5.9xlarge, ml.c5.xlarge, ml.c4.xlarge, ml.t2.2xlarge, ml.t2.medium, ml.c5.18xlarge, ml.p3.2xlarge, ml.m5.xlarge, ml.m4.10xlarge, ml.t2.large, ml.m5.12xlarge, ml.m4.xlarge, ml.m5.24xlarge, ml.m4.2xlarge, ml.p2.8xlarge, ml.m5.2xlarge, ml.p3.8xlarge, ml.m4.4xlarge]

In [39]:
from sagemaker.predictor import json_serializer, json_deserializer
sample_predictor = sagemaker.predictor.RealTimePredictor(
    endpoint=endpoint_name,
    content_type = 'application/json',
    serializer = json_serializer,
    deserializer = json_deserializer,
    sagemaker_session=sagemaker.Session(boto_session = boto3_session)
)

In [41]:
post_data = {'data': 'hogehogeaaa'}
sample_predictor.predict(post_data)

{'post_data': 'hogehogeaaa', 'result': 'output as model'}

In [42]:
sample_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint with name: knn-ml-m4-xlarge-1539653396-676969
